In [ ]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

"""
Hybrid Recommendation System for Product Images and Metadata

This script builds a hybrid recommendation system that leverages both image features
(using VGG16 pre-trained CNN) and text metadata (TF-IDF vectorization).
It combines these features to recommend similar products based on a given input product.

Key Features:
1. **Image Features**: Extracted using VGG16 pre-trained model.
2. **Text Metadata**: Processed using TF-IDF Vectorizer.
3. **Hybrid Similarity**: Weighted combination of image and text similarities.
4. **Visualization**: Displays input product and recommended products as images.

Dependencies:
- TensorFlow
- scikit-learn
- matplotlib
- pandas
- PIL
- tqdm
"""

# ----------------------------
# Step 1: Load Metadata
# ----------------------------
# Load the metadata file (CSV format)
data = pd.read_csv('your_metadata_file.csv')

# Drop unneeded columns (e.g., Product URL)
data = data.drop(['Product URL'], axis=1)

# ----------------------------
# Step 2: Image Feature Extraction
# ----------------------------
# Initialize VGG16 model for feature extraction
base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=base_model.output)

def preprocess_img(img_path):
    """
    Preprocess the input image to match VGG16 input requirements.
    Args:
        img_path (str): Path to the image file.
    Returns:
        np.array: Preprocessed image array.
    """
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded)

def extract_features(img_path):
    """
    Extract features from the image using VGG16.
    Args:
        img_path (str): Path to the image file.
    Returns:
        np.array: Flattened and normalized image features.
    """
    img = preprocess_img(img_path)
    features = model.predict(img)
    flattened_features = features.flatten()
    return flattened_features / np.linalg.norm(flattened_features)

# Extract image features for all images
image_features = []
image_paths = []
for img_path in tqdm(data['Image Path']):
    if os.path.exists(img_path):
        features = extract_features(img_path)
        image_features.append(features)
        image_paths.append(img_path)
    else:
        # Placeholder for missing images
        image_features.append(np.zeros((7 * 7 * 512)))

data['Image Features'] = image_features

# ----------------------------
# Step 3: Text Feature Extraction
# ----------------------------
# Combine textual metadata columns
vectorizer = TfidfVectorizer(stop_words='english')
text_data = data['Product Name'] + ' ' + data['Brand'] + ' ' + data['Category'].astype(str)
tfidf_matrix = vectorizer.fit_transform(text_data)

# ----------------------------
# Step 4: Hybrid Recommendation Function
# ----------------------------
def get_recommendations(input_index, data, tfidf_matrix, top_n=5):
    """
    Recommend similar products based on hybrid similarity (image + text).
    Args:
        input_index (int): Index of the input product.
        data (DataFrame): Product metadata and features.
        tfidf_matrix (sparse matrix): TF-IDF matrix of text metadata.
        top_n (int): Number of recommendations to display.
    """
    # Extract input features
    input_image_features = data['Image Features'][input_index]
    input_text_vector = tfidf_matrix[input_index]

    # Compute cosine similarities
    image_similarities = [1 - cosine(input_image_features, features) for features in data['Image Features']]
    text_similarities = cosine_similarity(input_text_vector, tfidf_matrix).flatten()

    # Combine similarities (weighted sum)
    combined_similarity = 0.6 * np.array(image_similarities) + 0.4 * text_similarities

    # Sort indices by similarity, excluding the input product itself
    similar_indices = np.argsort(combined_similarity)[::-1]
    similar_indices = [i for i in similar_indices if i != input_index]

    # Visualization
    fig, axes = plt.subplots(nrows=top_n + 1, ncols=1, figsize=(6, 10))
    if top_n == 1:
        axes = [axes]  # Ensure axes is iterable for single recommendation

    # Display input product
    img = Image.open(data['Image Path'][input_index])
    axes[0].imshow(img)
    axes[0].axis('off')
    axes[0].set_title(f"Input Product: {data['Product Name'][input_index]}\nPrice: {data['Offer Price'][input_index]}")

    # Display recommended products
    for i, idx in enumerate(similar_indices[:top_n]):
        img = Image.open(data['Image Path'][idx])
        axes[i + 1].imshow(img)
        axes[i + 1].axis('off')
        axes[i + 1].set_title(f"{data['Product Name'][idx]}\nPrice: {data['Offer Price'][idx]}")

    plt.tight_layout()
    plt.show()

# ----------------------------
# Step 5: Example Usage
# ----------------------------
# Recommend similar products for a given product (e.g., first product in the dataset)
get_recommendations(0, data, tfidf_matrix, top_n=5)